<a href="https://colab.research.google.com/github/stas420/machine_learning_ev_ice/blob/main/src/EV_to_ICE_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and dependencies
Run the below cell, which shall clone the repo contents and prepare the machine for work. This may take a few minutes.

***Important:*** This is the first cell you shall run in this project, otherwise it won't be working.

After 'pip' is done, it may ask you to restart this session - please do accordingly.

In [1]:
# remove possible leftovers and clear-clone the whole repo
%cd /content
%rm -rf ./machine_learning_ev_ice
!git clone https://github.com/stas420/machine_learning_ev_ice.git

# a git pull pro forma
%cd ./machine_learning_ev_ice
!git pull

# go to our main directory
%cd ./src

# check (and download, if needed) dependencies for the project
%pip install -r requirements.txt

/content
Cloning into 'machine_learning_ev_ice'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 92 (delta 48), reused 44 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (92/92), 25.59 MiB | 30.25 MiB/s, done.
Resolving deltas: 100% (48/48), done.
/content/machine_learning_ev_ice
Already up to date.
/content/machine_learning_ev_ice/src
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# mount data

poberamy dane z podmontowanego Google Drive

In [1]:
%cd /content

from google.colab import drive
import os

if not 'drive' in os.listdir():
  drive.mount('/content/drive')
else:
  print("drajw dajrektory alredi prezent")

/content
Mounted at /content/drive


### tu zmienne globalne

In [5]:
VIDEOS_PATH = '/content/drive/MyDrive/videos'
IMAGES_PATH = '/content/imgs'

# ekstrakcja ramek

*robić WiTW, gdy ich nie ma na dysku!*

In [3]:
# clearing leftovers pro forma
%rm -rf /content/imgs
%mkdir /content/imgs

import cv2
import os

# function which takes a .mp4 file and extracts its frames,
# then saves them as images in the provided directory
def extract_frames(video_path, output_dir, fps=1) -> None:
    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    count = 0
    saved = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        else:
            frame_path = os.path.join(output_dir, f"frame_{saved:03d}.jpg")
            cv2.imwrite(frame_path, frame)
            saved += 1
        count += 1
    cap.release()

# image extraction
for x in os.listdir(VIDEOS_PATH):
  extract_frames(VIDEOS_PATH + '/' + x, "/content/imgs/" + x, fps=1)

!ls -la ./content/imgs

In [10]:
# Extract frames for a time range and pack them into a zip archive
from typing import List
import datetime as dt
import zipfile

# EDIT HERE
START_DATE: dt.date = dt.date.fromisoformat("2024-03-01")
END_DATE: dt.date = dt.date.fromisoformat("2024-03-31")

def get_dates_in_range(start_date: dt.date, end_date: dt.date) -> List[str]:
  delta = end_date - start_date
  return [(start_date + dt.timedelta(days=i)).isoformat() for i in range(delta.days + 1)]

## Archives the given directories into a single zip file and returns the path to it
def zip_frame_dirs(start_date: dt.date, end_date: dt.date, output_dir = IMAGES_PATH) -> str:
  dir_names = [date + ".mp4" for date in get_dates_in_range(start_date, end_date)]
  existing_dir_paths = [output_dir + "/" + name for name in dir_names if name in os.listdir(output_dir)]
  zip_filename = f"{start_date.isoformat()}--{end_date.isoformat()}.zip"
  with zipfile.ZipFile(output_dir + "/" + zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for dir_path in existing_dir_paths:
            for root, _, files in os.walk(dir_path):
                for file in files:
                    full_path = os.path.join(root, file)
                    arcname = os.path.relpath(full_path, start=os.path.dirname(dir_path))
                    print(f"Writing {full_path} to zip file")
                    zipf.write(full_path, arcname)
  return zip_filename

if __name__ == "__main__":
  file_names = [date + ".mp4" for date in get_dates_in_range(START_DATE, END_DATE)]
  file_names_in_dir = [name for name in file_names if name in os.listdir(VIDEOS_PATH)]
  for name in file_names_in_dir:
    if os.path.isdir(IMAGES_PATH + "/" + name):
      print(f"Directory {name} already exists, skipping")
      continue
    print(f"Extracting from {name}")
    extract_frames(VIDEOS_PATH + "/" + name, IMAGES_PATH + "/" + name)
    print(f"Done with {name}")
  print(f"All videos in range {START_DATE.isoformat()} - {END_DATE.isoformat()} done extracting")
  zip_filename = zip_frame_dirs(START_DATE, END_DATE)
  print(f"Done zipping files to {zip_filename}")

Directory 2024-03-01.mp4 already exists, skipping
Directory 2024-03-02.mp4 already exists, skipping
Directory 2024-03-03.mp4 already exists, skipping
Directory 2024-03-04.mp4 already exists, skipping
Directory 2024-03-05.mp4 already exists, skipping
Directory 2024-03-06.mp4 already exists, skipping
Directory 2024-03-07.mp4 already exists, skipping
Directory 2024-03-08.mp4 already exists, skipping
Directory 2024-03-09.mp4 already exists, skipping
Directory 2024-03-10.mp4 already exists, skipping
Directory 2024-03-11.mp4 already exists, skipping
Directory 2024-03-12.mp4 already exists, skipping
Directory 2024-03-13.mp4 already exists, skipping
Directory 2024-03-14.mp4 already exists, skipping
Directory 2024-03-15.mp4 already exists, skipping
Directory 2024-03-16.mp4 already exists, skipping
Directory 2024-03-17.mp4 already exists, skipping
Directory 2024-03-18.mp4 already exists, skipping
Directory 2024-03-19.mp4 already exists, skipping
Directory 2024-03-20.mp4 already exists, skipping


# konwersja XML do formatu YOLO

In [ ]:
from xml.dom import minidom
import os
import glob

LUT = {"class0": 0, "class1": 1, "class2": 2}

def convert_coordinates(size, box):
    dw = 1.0/size[0]
    dh = 1.0/size[1]
    x = (box[0]+box[1])/2.0
    y = (box[2]+box[3])/2.0
    w = box[1]-box[0]
    h = box[3]-box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)


def convert_xml2yolo():

    for fname in glob.glob("data/*.xml"):
        xmldoc = minidom.parse(fname)
        fname_out = (fname[:-4]+'.txt')

        with open(fname_out, "w") as f:

            itemlist = xmldoc.getElementsByTagName('object')
            size = xmldoc.getElementsByTagName('size')[0]
            width = int((size.getElementsByTagName('width')[0]).firstChild.data)
            height = int((size.getElementsByTagName('height')[0]).firstChild.data)

            for item in itemlist:
                classid =  (item.getElementsByTagName('name')[0]).firstChild.data
                if classid in LUT:
                    label_str = str(LUT[classid])
                else:
                    label_str = "-1"
                    print ("warning: label '%s' not in look-up table" % classid)

                xmin = ((item.getElementsByTagName('bndbox')[0]).getElementsByTagName('xmin')[0]).firstChild.data
                ymin = ((item.getElementsByTagName('bndbox')[0]).getElementsByTagName('ymin')[0]).firstChild.data
                xmax = ((item.getElementsByTagName('bndbox')[0]).getElementsByTagName('xmax')[0]).firstChild.data
                ymax = ((item.getElementsByTagName('bndbox')[0]).getElementsByTagName('ymax')[0]).firstChild.data
                b = (float(xmin), float(xmax), float(ymin), float(ymax))
                bb = convert_coordinates((width,height), b)

                f.write(label_str + " " + " ".join([("%.6f" % a) for a in bb]) + '\n')

        print ("%s" % fname_out)


if __name__ == '__main__':
    convert_xml2yolo()

# tutaj model


In [ ]:
%cd /content/machine_learning_ev_ice/src

# for development only
%rm -rf /content/results
from ultralytics import YOLO
import cv2
import os

def is_green_plate(plate_img):
    hsv = cv2.cvtColor(plate_img, cv2.COLOR_BGR2HSV)
    h, s, _ = hsv.mean(axis=0).mean(axis=0)
    return 35 < h < 85 and s > 40

model = YOLO('./yolov8s.pt')
input_dir = "/content/imgs/2024-02-08.mp4"
output_dir = "/content/results"
os.makedirs(output_dir, exist_ok=True)

# for each file, check if it's a jpg and pass it to the model
for fname in os.listdir(input_dir):
    if not fname.endswith(".jpg"):
        continue
    path = os.path.join(input_dir, fname)
    img = cv2.imread(path)
    results = model(img)

    # for each 'box' in obtained result write a frame around the found object
    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        crop = img[y1:y2, x1:x2]
        class_id = int(box.cls)
        label = model.names[class_id]
        color = (0,255,0) # if label == "green" else (255,0,0)

        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imwrite(os.path.join(output_dir, fname), img)


/content/machine_learning_ev_ice/src

0: 384x640 8 cars, 1 bus, 1 truck, 663.0ms
Speed: 4.1ms preprocess, 663.0ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 1 truck, 643.1ms
Speed: 4.0ms preprocess, 643.1ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 3 buss, 1 truck, 630.1ms
Speed: 5.1ms preprocess, 630.1ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 1 bus, 1 truck, 633.0ms
Speed: 4.4ms preprocess, 633.0ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 cars, 1 bus, 2 trucks, 1 traffic light, 489.0ms
Speed: 6.4ms preprocess, 489.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 16 cars, 1 truck, 1 traffic light, 414.2ms
Speed: 4.7ms preprocess, 414.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 416.3ms
Speed: 4.3ms preprocess, 416.3ms inference, 1.4ms postpro

Dżem dobry
:)